In [22]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import librosa

import soundfile as sf

import sys
sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

In [3]:
fs = 16000
ckpt_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/wavlm/WavLM-Large.pt"
path = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/HKK/wav/arctic_a0001.wav"

In [36]:
def read_audio(fname, fs=16000):
    """ Load an audio file and return PCM along with the sample rate """

    wav, sr = sf.read(fname)
    if sr!=fs:
        wav, _ = librosa.load(fname, fs)
        sf.write("temp.wav", wav, fs, subtype="PCM_24") # 書き込み
        wav, sr = sf.read("temp.wav")
    assert sr == fs

    return wav, fs

In [8]:
class PretrainedWav2VecModel(nn.Module):
    def __init__(self, fname):
        super().__init__()

        model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([fname])
        model = model[0]
        model.eval()

        self.model = model

    def forward(self, x):
        with torch.no_grad():
            z = self.model.feature_extractor(x)
            if isinstance(z, tuple):
                z = z[0]
            c = self.model.feature_aggregator(z)
        return z, c

In [9]:
class Prediction:
    """ Lightweight wrapper around a fairspeech embedding model """

    def __init__(self, fname, gpu=0):
        self.gpu = gpu
        self.model = PretrainedWav2VecModel(fname).cuda(gpu)

    def __call__(self, x):
        x = torch.from_numpy(x).float().cuda(self.gpu)
        with torch.no_grad():
            z, c = self.model(x.unsqueeze(0))

        return z.squeeze(0).cpu().numpy(), c.squeeze(0).cpu().numpy()

In [41]:
# model = Prediction(ckpt_path, 0)
wav, sr = read_audio(path)
# z, c = model(wav)